In [1]:
from PIL import Image;
import numpy as np;
import operator;
import matplotlib.pyplot as plt;
import random;
import math;
import statistics;
import os;
import time;
import xlwt;
from xlwt import Workbook;
import openpyxl;

In [2]:
wb = Workbook();
sheet1 = wb.add_sheet('Sheet 1');

In [3]:
sheet1.write(0, 0, 'PAPER_ID')
sheet1.write(0, 1, 'TC_ID')
for i in range(1, 31, 1):
    sheet1.write(0, 2*i, "ITR_"+str(i))
    sheet1.write(0, 2*i+1, "TIME_"+str(i))
    
sheet1.write(0, 62, 'Total_Time');
sheet1.write(0, 63, 'BEST_ITR');
sheet1.write(0, 64, 'Max Similarity');
sheet1.write(0, 65, 'Image_Pre_Time');
wb.save('DOC.xls')

In [4]:
source_files = []
source_dirs = []

rootdir = 'C:/Users/parma/ML_Tutorials/Image_Similarity/Source'

for subdir, dirs, files in os.walk(rootdir):
    if(len(files) == 0): 
        for i in range(0, len(dirs), 1):
            source_dirs.append(dirs[i]);
        continue;
    
    fs = []
    for i in range(0, len(files), 1):
        fs.append(files[i]);
    source_files.append(fs);

In [5]:
copy_files = []
copy_dirs = []

rootdir = 'C:/Users/parma/ML_Tutorials/Image_Similarity/Copy'

for subdir, dirs, files in os.walk(rootdir):
    if(len(files) == 0): 
        for i in range(0, len(dirs), 1):
            copy_dirs.append(dirs[i]);
        continue;
    
    fs = []
    for i in range(0, len(files), 1):
        fs.append(files[i]);
    copy_files.append(fs);

In [6]:
D = (255**2)*3;

def dist(PIX_A, PIX_B):
    d = (PIX_A[0] - PIX_B[0])**2 + (PIX_A[1] - PIX_B[1])**2 + (PIX_A[2] - PIX_B[2])**2;
    return math.sqrt(d/D);

In [7]:
random.seed(0)

In [8]:
# for DIR_ID in range(0, len(source_dirs), 1):
#     for TC_ID in range(0, len(source_files[DIR_ID]), 1):
#         print(source_dirs[DIR_ID], source_files[DIR_ID][TC_ID], end = " ||| ")
#         print("(", copy_dirs[DIR_ID], copy_files[DIR_ID][TC_ID], ")")

In [9]:
def fetch_start_points(test_image, test_pix):
    spread = 3;

    start_points = []

    for i in range(0, test_image.size[0], 1):
        for j in range(0, test_image.size[1], 1):
            dissimilarity = 0;
            ct = 0;
            for k in range(i-spread, i+spread+1, 1):
                for l in range(j-spread, j+spread+1, 1):
                    if(k == i and j == l):
                        continue;
                    if(k < 0 or k >= test_image.size[0] or l < 0 or l >= test_image.size[1]):
                        continue;
                    dissimilarity += dist(test_pix[k, l], test_pix[i, j]);

            Diss = dissimilarity;
            Diss = round(Diss, 2)
            start_points.append([Diss, i, j]);

    start_points = sorted(start_points, key = operator.itemgetter(0));
    start_points.reverse()
    
    return start_points;

In [10]:
def aggregate_score(scores):
    random.shuffle(scores);
    total_buckets = 100;
    max_bucket_size = len(scores)/total_buckets;
    buckets = [];
    curr_size = 0;
    curr_sum = 0;
    for sc in scores:
        curr_size += 1;
        curr_sum += sc;
        if(curr_size == max_bucket_size):
            buckets.append(curr_sum/curr_size);
            curr_sum = 0;
            curr_size = 0;
    if(curr_size > 0):
        buckets.append(curr_sum/curr_size);
    
    return statistics.median(buckets);

In [11]:
class Path:
    def __init__(self):
        self.points = [];
        self.score = [];
        self.agg_score = -1;

In [12]:
curr_row = 1;

In [13]:
for DIR_ID in range(3, len(source_dirs), 1):
    paper_id = DIR_ID + 1;
#     sheet1.write(curr_row, 0, paper_id);
    
    for TC_ID in range(0, len(source_files[DIR_ID]), 1):
#         if(TC_ID > 0):
#             break;
        tc_id = TC_ID + 1;
        print(paper_id, tc_id);
        sheet1.write(curr_row, 1, tc_id);
        sheet1.write(curr_row, 0, paper_id);
        
        SA = time.time();
        
        Folder = source_dirs[DIR_ID]
        Source_File = source_files[DIR_ID][TC_ID];
        Test_File = copy_files[DIR_ID][TC_ID];
        
        source_dir = "Source/"+Folder+"/"+Source_File;
        source_image = Image.open(source_dir);
        source_pix = source_image.load();
        
        test_dir = "Copy/"+Folder+"/"+Test_File;
        test_image = Image.open(test_dir);
        test_pix = test_image.load();
        
        start = time.time()
        start_points = fetch_start_points(test_image, test_pix);
        end = time.time()
        
        sheet1.write(curr_row, 65, round(end - start, 3));
        
        ans = Path();
        score_on_itr = -1;
        
#         MAX_STEP = math.floor(min(test_image.size[0], test_image.size[1])/10);
        MAX_STEP = math.floor((test_image.size[0] + test_image.size[1])/20);
        
        RANDOM_WALK_LEN = 3000;
        
        ITR_LIMIT = 30;
                
        for START in range(0, ITR_LIMIT, 1):
#             print("ITR: ", START+1);
            start = time.time();
            
            start_X = start_points[START][1];
            start_Y = start_points[START][2];

            v = []
            for i in range(0, source_image.size[0], 1):
                for j in range(0, source_image.size[1], 1):
                    d = dist(test_pix[start_X, start_Y], source_pix[i,j]);
                    v.append([d, i, j]);

            v = sorted(v, key = operator.itemgetter(0));
            v = v[:100]

            paths = [];
            for i in range(0, 100, 1):
                p = Path();
                p.points.append([v[i][1], v[i][2]]);
                p.score.append(v[i][0]);
                paths.append(p);

            curr_X = start_X;
            curr_Y = start_Y;

            for itr in range(0, RANDOM_WALK_LEN, 1):
                step_X = random.randint(1, MAX_STEP);
                step_Y = random.randint(1, MAX_STEP);

                off_X = random.randint(0, 1);
                off_Y = random.randint(0, 1);

                if(off_X == 0): off_X = -1;
                if(off_Y == 0): off_Y = -1;

                curr_X = curr_X + off_X * step_X;
                curr_Y = curr_Y + off_Y * step_Y;

                if(curr_X < 0 or curr_X >= test_image.size[0] or curr_Y < 0 or curr_Y >= test_image.size[1]):
                    curr_X = curr_X - off_X * step_X;
                    curr_Y = curr_Y - off_Y * step_Y;
                    continue;

                img_RGB = test_pix[curr_X, curr_Y];

                for i in range(0, len(paths), 1):
                    if(paths[i].score[-1] == -1):continue;
                    last_X = paths[i].points[-1][0];
                    last_Y = paths[i].points[-1][1];

                    new_X = last_X + off_X * step_X;
                    new_Y = last_Y + off_Y * step_Y;

                    if(new_X < 0 or new_X >= source_image.size[0] or new_Y == 0 or new_Y >= source_image.size[1]):
                        paths[i].score.append(-1);
                        continue;

                    d = dist(img_RGB, source_pix[new_X, new_Y]);

                    paths[i].score.append(d);
                    paths[i].points.append([new_X, new_Y]);
            
            best_itr_score = Path();
            
            for p in paths:
                if(p.score[-1] == -1):
                    continue;

                p.agg_score = aggregate_score(p.score);
                if(p.agg_score < best_itr_score.agg_score or best_itr_score.agg_score == -1):
                    best_itr_score = p;
                if(p.agg_score < ans.agg_score or ans.agg_score == -1):
                    ans = p;
                    score_on_itr = START+1;
            
            end = time.time();
            sc = best_itr_score.agg_score;
            if(sc != -1):
                sc = 1 - sc;
            
            sheet1.write(curr_row, 2*(START+1), round(sc, 3));
            sheet1.write(curr_row, 2*(START+1)+1, round(end-start, 3));
        
        output_dir = "Results/" + Folder + "/result_"+ Test_File;
        for i in range(0, len(ans.points), 1):
            source_pix[ans.points[i][0], ans.points[i][1]] = (0, 0, 0)

        source_image.save(output_dir)
        SE = time.time();
        
        sheet1.write(curr_row, 62, round(SE-SA, 3));
        sheet1.write(curr_row, 63, score_on_itr);
        sheet1.write(curr_row, 64, round(1-ans.agg_score, 3));
        curr_row += 1;

4 1


In [14]:
wb.save('DOC.xls')

In [15]:
# TC = "009"

In [16]:
# Folder = "002"

In [17]:
# source_dir = "Source/"+Folder+"/source_" + TC + ".JPG"
# source_image = Image.open(source_dir)
# source_pix = source_image.load()

In [18]:
# test_dir = "Copy/"+Folder+"/copy_" + TC + ".JPG"
# test_image = Image.open(test_dir)
# test_pix = test_image.load()

In [19]:
# source_image.size, test_image.size

In [20]:
# spread = 5;

# start_points = []

# for i in range(0, test_image.size[0], 1):
#     for j in range(0, test_image.size[1], 1):
#         dissimilarity = 0;
#         ct = 0;
#         for k in range(i-spread, i+spread+1, 1):
#             for l in range(j-spread, j+spread+1, 1):
#                 if(k == i and j == l):
#                     continue;
#                 if(k < 0 or k >= test_image.size[0] or l < 0 or l >= test_image.size[1]):
#                     continue;
#                 dissimilarity += dist(test_pix[k, l], test_pix[i, j]);
        
#         Diss = dissimilarity;
#         Diss = round(Diss, 2)
#         start_points.append([Diss, i, j]);

# start_points = sorted(start_points, key = operator.itemgetter(0));
# start_points.reverse()

In [21]:
# start_points[0]

In [22]:
# len(start_points)

In [23]:
# LMT = 100;
# x = np.arange(0, LMT, 1)
# y = [];
# for i in range(0, LMT, 1):
#     y.append(start_points[i][0])
# plt.plot(x, y)

In [24]:
# start_points = start_points[:100]
# len(start_points)

In [25]:
# fig = plt.figure(figsize = (10,10)) 
# plt.imshow( matrix , cmap = 'copper' , interpolation = 'nearest')
# plt.show()

In [26]:
# class Path:
#     def __init__(self):
#         self.points = [];
#         self.score = [];
#         self.agg_score = -1;

In [27]:
# def aggregate_score(scores):
#     random.shuffle(scores);
#     total_buckets = 100;
#     max_bucket_size = len(scores)/total_buckets;
#     buckets = [];
#     curr_size = 0;
#     curr_sum = 0;
#     for sc in scores:
#         curr_size += 1;
#         curr_sum += sc;
#         if(curr_size == max_bucket_size):
#             buckets.append(curr_sum/curr_size);
#             curr_sum = 0;
#             curr_size = 0;
#     if(curr_size > 0):
#         buckets.append(curr_sum/curr_size);
    
#     return statistics.median(buckets);

In [28]:
# ans = Path();

In [29]:
# MAX_STEP = math.floor(min(test_image.size[0], test_image.size[1])/10);
# MAX_STEP = math.floor((test_image.size[0] + test_image.size[1])/20);

In [30]:
# MAX_STEP

In [31]:
# RANDOM_WALK_LEN = 5000;

In [32]:
# ITR_LIMIT = 10;

# for START in range(0, ITR_LIMIT, 1):
#     print("ITR: ", START+1);
    
#     start_X = start_points[START][1];
#     start_Y = start_points[START][2];

#     v = []
#     for i in range(0, source_image.size[0], 1):
#         for j in range(0, source_image.size[1], 1):
#             d = dist(test_pix[start_X, start_Y], source_pix[i,j]);
#             v.append([d, i, j]);

#     v = sorted(v, key = operator.itemgetter(0));
#     v = v[:100]
    
#     paths = [];
#     for i in range(0, 100, 1):
#         p = Path();
#         p.points.append([v[i][1], v[i][2]]);
#         p.score.append(v[i][0]);
#         paths.append(p);
    
#     curr_X = start_X;
#     curr_Y = start_Y;

#     for itr in range(0, RANDOM_WALK_LEN, 1):
#         step_X = random.randint(1, MAX_STEP);
#         step_Y = random.randint(1, MAX_STEP);
        
#         off_X = random.randint(0, 1);
#         off_Y = random.randint(0, 1);

#         if(off_X == 0): off_X = -1;
#         if(off_Y == 0): off_Y = -1;

#         curr_X = curr_X + off_X * step_X;
#         curr_Y = curr_Y + off_Y * step_Y;

#         if(curr_X < 0 or curr_X >= test_image.size[0] or curr_Y < 0 or curr_Y >= test_image.size[1]):
#             curr_X = curr_X - off_X * step_X;
#             curr_Y = curr_Y - off_Y * step_Y;
#             continue;

#         img_RGB = test_pix[curr_X, curr_Y];

#         for i in range(0, len(paths), 1):
#             if(paths[i].score[-1] == -1):continue;
#             last_X = paths[i].points[-1][0];
#             last_Y = paths[i].points[-1][1];

#             new_X = last_X + off_X * step_X;
#             new_Y = last_Y + off_Y * step_Y;

#             if(new_X < 0 or new_X >= source_image.size[0] or new_Y == 0 or new_Y >= source_image.size[1]):
#                 paths[i].score.append(-1);
#                 continue;

#             d = dist(img_RGB, source_pix[new_X, new_Y]);

#             paths[i].score.append(d);
#             paths[i].points.append([new_X, new_Y]);
        
#     final_P = Path();
#     for p in paths:
#         if(p.score[-1] == -1):
#             continue;

#         p.agg_score = aggregate_score(p.score);
#         if(p.agg_score < final_P.agg_score or final_P.agg_score == -1):
#             final_P = p;

#     if(final_P.agg_score == -1):
#         continue;

#     if(final_P.agg_score < ans.agg_score or ans.agg_score == -1):
#         ans = final_P;

In [33]:
# LMT = 100;
# x = np.arange(0, LMT, 1)
# y = [];
# for i in range(0, LMT, 1):
#     y.append(v[i][0])
# plt.plot(x, y)

In [34]:
# output_dir = "New_Results/" + Folder + "/result_"+ TC + ".jpg";
# for i in range(0, len(ans.points), 1):
# #     print(final_P.points[i]);
#     source_pix[ans.points[i][0], ans.points[i][1]] = (0, 0, 0)

# source_image.save(output_dir)

In [35]:
# ans.agg_score

In [36]:
# 1 - ans.agg_score